In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
x_train,x_test=digits.data[:600], digits.data[:600]
y_train,y_test=digits.target[:600],digits.target[:600] #显然这里有一个很大的问题，训练集不可以和测试集一样
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(600, 64)
(600, 64)
(600,)
(600,)


In [2]:
import numpy as np
from PIL import Image
def img_show(img):
    pil_img=Image.fromarray(np.uint8(img))
    pil_img.show()

In [3]:
img=x_train[0]
label=y_train[0]
print(label)

0


In [4]:
print(img.shape)

(64,)


In [5]:
img=img.reshape(8,8)#把图像形状变成原来的尺寸，即8*8
print(img.shape)

(8, 8)


In [7]:
img_show(img)

In [10]:
def img_show_2(img):
    img=(img/16)*255
    img=img.astype(np.uint8)
    pil_img=Image.fromarray(img)
    pil_img.show()

In [11]:
img_show_2(img)

In [ ]:
digits = load_digits()
x_train, x_test = digits.data[:600],digits.data[600:]
y_train,y_test = digits.target[:600],digits.data[600:] #这个应该是digits.target[]
# 注意训练集和测试集的数据需要不一样
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(600, 64)
(1197, 64)
(600,)
(1197, 64)


In [14]:
digits = load_digits()
x_train, x_test = digits.data[600:],digits.data[:600]
y_train,y_test = digits.target[600:],digits.target[:600]
# 注意训练集和测试集的比例最好 7：3， 8：2
print(x_train.shape)
print(x_test.shape) 
print(y_train.shape)
print(y_test.shape)

(1197, 64)
(600, 64)
(1197,)
(600,)


In [15]:
print(x_train[0])
print(y_train[0])

[ 0.  0. 10. 15.  2.  0.  0.  0.  0.  7. 16. 16.  6.  0.  0.  0.  0. 12.
 13. 12.  9.  0.  0.  0.  0.  8.  9. 13.  7.  0.  0.  0.  0.  0.  0. 16.
  5.  0.  0.  0.  0.  0.  6. 15.  1.  0.  0.  0.  0.  0. 16. 14.  4.  5.
  8.  3.  0.  0.  8. 16. 16. 16. 16.  9.]
2


可以发现x的取值范围是[0,16],y的取值范围是[0,9]

In [16]:
#归一化
x_train = x_train /16
x_test = x_test /16
print(x_train[0])

[0.     0.     0.625  0.9375 0.125  0.     0.     0.     0.     0.4375
 1.     1.     0.375  0.     0.     0.     0.     0.75   0.8125 0.75
 0.5625 0.     0.     0.     0.     0.5    0.5625 0.8125 0.4375 0.
 0.     0.     0.     0.     0.     1.     0.3125 0.     0.     0.
 0.     0.     0.375  0.9375 0.0625 0.     0.     0.     0.     0.
 1.     0.875  0.25   0.3125 0.5    0.1875 0.     0.     0.5    1.
 1.     1.     1.     0.5625]


In [17]:
def to_one_hot(labels,num_classes=10):
    one_hot=np.zeros(len(labels),num_classes)
    one_hot[np.arrange(len(labels)),labels]=1
    return one_hot
    #one_hot is a 2D array（一个矩阵简单理解）
y_train_one_hot=to_one_hot(y_train)
y_test_one_hot=to_one_hot(y_test)
print(y_train_one_hot[0])

TypeError: Cannot interpret '10' as a data type

In [19]:
def to_one_hot(labels,num_classes=10):
    one_hot=np.zeros((len(labels),num_classes))
    one_hot[np.arange(len(labels)),labels]=1
    return one_hot
    #one_hot is a 2D array（一个矩阵简单理解）
y_train_one_hot=to_one_hot(y_train)
y_test_one_hot=to_one_hot(y_test)
print(y_train_one_hot[0])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [1]:
#初始化神经网络参数
def init_parameters(input_size,hidden_size,output_size):
    W1=np.random.randn(hidden_size,input_size)*0.01
    # b1=np.zeros(tuple)
    b1=np.zeros((hidden_size,1))
    W2=np.random.randn(output_size,hidden_size)*0.01
    b2=np.zeros((output_size,1))
    return{"W1": W1, "b1": b1, "W2": W2, "b2": b2 }


In [2]:
#设定激活函数
def sigmoid(z):
    return 1/(1+np.exp(-z))
#激活函数的导数
def sigmoid_derivative(z):
    return sigmoid(z)*(1-sigmoid(z))

In [3]:
#定义前向传播
def forward_propa(X,parameters):
    W1,b1,W2,b2=parameters["W1"], parameters["b1"], parameters["W2"], parameters["b2"]
    Z1=np.dot(W1,X.T)+b1
    A1=sigmoid(Z1)
    Z2=np.dot(W2,A1)+b2
    A2=sigmoid(Z2)
    return {"Z1":Z1,"A1":A1, "Z2":Z2, "A2":A2}

In [4]:
#定义损失函数
def compute_loss(A2,Y):
    m=Y.shape[0]
    loss=-np.sum(Y*np.log(A2)+(1-Y)*np.log(1-A2))/m
    return np.squeeze(loss) # 返回一个标量


In [ ]:
#反向传播
def backward_propa(X,Y,parameters,cache):
    m=X.shape[0]